In [1]:
import requests
from bs4 import BeautifulSoup
from bs4.element import ResultSet, Tag
import pandas as pd
from tabulate import tabulate
import csv
import time

In [3]:

base_url = "https://www.etoos.com/teacher/Evaluate/default.asp?"
teacher_id_list = [200439, 200245]
page_range = 44
# -> 총 데이터 개수 = 선생 수 x 페이지 범위 x 한 페이지 당 개수 = 2 x 34 x 15 = 1020
teacher_prefix = "TEACHER_ID="
page_prefix = "&page="
path = "./data/"
f_name = 'review.csv'
csv_full_name = path + f_name

def getFullUrl(teacher_id:int, page:int)->str:
    global base_url
    url = base_url + teacher_prefix + str(teacher_id) + page_prefix + str(page)
    return url

review_list = []

for teacher_id in teacher_id_list:
    for page in range(1,page_range + 1):
        print("curr page:", page)
        # 너무 자주보내면 응답을 안 줌
        time.sleep(2)
        response = requests.get(getFullUrl(teacher_id=teacher_id, page=page))
        # 응답 성공
        if response.status_code == 200:
            # 한글 깨짐 방지 인코딩
            response.encoding = 'euc-kr'
            soup = BeautifulSoup(response.text, 'html.parser')
            # 첫 페이지면 선생님 이름 가져오기
            if page == 1:
                teacher_name = soup.select_one("div#TeacherLeftGnbEach a.tit").get_text(strip=True)
                print(teacher_name)
            # 리뷰가 적힌 행들 가져오기
            rows = soup.select('tbody > tr')
            for row in rows:
                # 등록 번호 찾기
                reg_num = row.select_one('td:nth-child(1)').get_text(strip=True)
                # BEST 수강 후기는 매 페이지마다 표시되므로 스킵
                if reg_num == "BEST": continue
                # 리뷰 내용 찾기
                review_text = row.select_one('td.wr_tit p.tit')
                ## 쓸데없는 정보 제거( new! 표시 )
                if review_text.span != None:
                    review_text.span.decompose()
                review_text = review_text.text.strip()
                # 등록 날짜 찾기
                date = row.select_one('td:nth-child(5)').get_text(strip=True)
                # 한 행 출력
                print(teacher_name, reg_num, review_text, date)
                # csv로 저장할 행 생성
                review = [teacher_name, reg_num, review_text, date]
                # 전체 리뷰에 추가
                review_list.append(review)


curr page: 1
주혜연 선생님
주혜연 선생님 647 정말 만족하는 강좌인 것 같아요ㅜ 2025.01.14
주혜연 선생님 646 구멍 매꾸기 위해 들어야하는 필수 강의 2025.01.14
주혜연 선생님 645 항상 영어 3등급인데 얻어가는거 많아요 2025.01.12
주혜연 선생님 644 처음으로 영어 지문이 읽혀요 2025.01.12
주혜연 선생님 643 쭈쌤돠 함께하면 영어 성적도 상승! 2025.01.12
주혜연 선생님 642 듣기를 풀면서 독해 2025.01.09
주혜연 선생님 641 중학교 내용을 이정도로 압축? 2025.01.09
주혜연 선생님 640 지루하지 않은 단어암기 2025.01.07
주혜연 선생님 639 기출의 알고리즘 2025.01.06
주혜연 선생님 638 정말 고퀄이네요 2025.01.03
주혜연 선생님 637 어법에 자신이 생겼어요 2025.01.01
주혜연 선생님 636 노베 탈출 2024.12.22
주혜연 선생님 635 코드독해 2024.12.22
주혜연 선생님 634 새싹 모의고사 2024.12.22
주혜연 선생님 633 감사합니다! 2024.12.21
curr page: 2
주혜연 선생님 632 도움 많이 된 강좌 2024.12.20
주혜연 선생님 631 또 한 번의 빈칸을 마치며 2024.12.17
주혜연 선생님 630 주혜연 선생님 덕에 2등급 상승했습니다! 2024.12.09
주혜연 선생님 629 영어 기초공사 2024.11.30
주혜연 선생님 628 선생님 사랑합니다 2024.11.22
주혜연 선생님 627 커리어하이!!! 2024.11.20
주혜연 선생님 626 감사합니다. 2024.11.15
주혜연 선생님 625 뚜렷한 정답률 상승을 경험하게되었습니다! 2024.11.13
주혜연 선생님 624 완강 ! 2024.11.10
주혜연 선생님 623 영어 6등급인데 이거 듣고 1등급 됨 2024.11.06
주혜연 선생님 622 영어 노베이지만 너무 좋았습니다! 2024.11.04
주혜연 선생님 621 코드독해 완강했

In [5]:
# 파일로 저장
with open(csv_full_name, "w", encoding="utf-8", newline="") as csv_file:
    writer = csv.writer(csv_file)
    # 헤더 작성
    writer.writerow(["teacher_name", "no", "text", "date"])
    # 데이터 작성
    writer.writerows(review_list)

In [5]:
from konlpy.tag import Okt
okt = Okt()

In [2]:
df = pd.read_csv('./data/review.csv')
df

,teacher_name,no,text,date
0,주혜연 선생님,647,정말 만족하는 강좌인 것 같아요ㅜ,2025.01.14
1,주혜연 선생님,646,구멍 매꾸기 위해 들어야하는 필수 강의,2025.01.14
2,주혜연 선생님,645,항상 영어 3등급인데 얻어가는거 많아요,2025.01.12
3,주혜연 선생님,644,처음으로 영어 지문이 읽혀요,2025.01.12
4,주혜연 선생님,643,쭈쌤돠 함께하면 영어 성적도 상승!,2025.01.12
...,...,...,...,...
1302,강원우 선생님,82,이것은 혁명이다요,2019.07.12
1303,강원우 선생님,81,이해하기 쉽네요,2019.07.02
1304,강원우 선생님,80,강의명에 맞는 강좌!,2019.06.30
1305,강원우 선생님,79,해석이 정확하게되요,2019.06.20


In [3]:

import re 

# 정규식 적용 함수
def clean_text(text):
    return re.sub("[^가-힣a-zA-Z0-9]+", " ", text)

# DataFrame의 'text' 열에 정규식 함수 적용
df['cleaned_text'] = df['text'].apply(clean_text)

# 결과 출력
print(df)

     teacher_name   no                   text        date  \
0         주혜연 선생님  647     정말 만족하는 강좌인 것 같아요ㅜ  2025.01.14   
1         주혜연 선생님  646  구멍 매꾸기 위해 들어야하는 필수 강의  2025.01.14   
2         주혜연 선생님  645  항상 영어 3등급인데 얻어가는거 많아요  2025.01.12   
3         주혜연 선생님  644        처음으로 영어 지문이 읽혀요  2025.01.12   
4         주혜연 선생님  643    쭈쌤돠 함께하면 영어 성적도 상승!  2025.01.12   
...           ...  ...                    ...         ...   
1302      강원우 선생님   82              이것은 혁명이다요  2019.07.12   
1303      강원우 선생님   81               이해하기 쉽네요  2019.07.02   
1304      강원우 선생님   80            강의명에 맞는 강좌!  2019.06.30   
1305      강원우 선생님   79             해석이 정확하게되요  2019.06.20   
1306      강원우 선생님   78           어려운 문장도 시원하게  2019.04.09   

               cleaned_text  
0        정말 만족하는 강좌인 것 같아요   
1     구멍 매꾸기 위해 들어야하는 필수 강의  
2     항상 영어 3등급인데 얻어가는거 많아요  
3           처음으로 영어 지문이 읽혀요  
4       쭈쌤돠 함께하면 영어 성적도 상승   
...                     ...  
1302              이것은 혁명이다요  
1303               이해하기 쉽네요

In [13]:
import re
from konlpy.tag import Okt

# Okt 객체 생성
okt = Okt()

# 정규식 적용 및 어근 추출 함수
def clean_and_tokenize(text):
    # 띄어쓰기 기준으로 토큰화
    tokens = text.split()
    
    # 각 토큰에 대해 정규식 적용
    cleaned_tokens = [re.sub("[^가-힣a-zA-Z0-9]+", " ", token) for token in tokens]
    
    # 공백 토큰 제거
    cleaned_tokens = [token.strip() for token in cleaned_tokens if token.strip()]
    
    # 어근 추출 (동사, 형용사, 명사, 부사)
    stems = []
    for token in cleaned_tokens:
        # 품사 태깅 및 어근 추출
        pos = okt.pos(token, stem=True)
        for word, tag in pos:
            if tag in ['Verb', 'Adjective', 'Noun', 'Adverb']:  # 동사, 형용사, 명사, 부사만 어간 추출
                stems.append(word)  # 어간 추가
    
    return stems


# DataFrame의 'text' 열에 정규식과 어근 추출 함수 적용
df['cleaned_and_stemmed'] = df['text'].apply(clean_and_tokenize)

# 결과 출력
print(df)

     teacher_name   no                   text        date  \
0         주혜연 선생님  647     정말 만족하는 강좌인 것 같아요ㅜ  2025.01.14   
1         주혜연 선생님  646  구멍 매꾸기 위해 들어야하는 필수 강의  2025.01.14   
2         주혜연 선생님  645  항상 영어 3등급인데 얻어가는거 많아요  2025.01.12   
3         주혜연 선생님  644        처음으로 영어 지문이 읽혀요  2025.01.12   
4         주혜연 선생님  643    쭈쌤돠 함께하면 영어 성적도 상승!  2025.01.12   
...           ...  ...                    ...         ...   
1302      강원우 선생님   82              이것은 혁명이다요  2019.07.12   
1303      강원우 선생님   81               이해하기 쉽네요  2019.07.02   
1304      강원우 선생님   80            강의명에 맞는 강좌!  2019.06.30   
1305      강원우 선생님   79             해석이 정확하게되요  2019.06.20   
1306      강원우 선생님   78           어려운 문장도 시원하게  2019.04.09   

               cleaned_text               cleaned_and_stemmed  
0        정말 만족하는 강좌인 것 같아요              [정말, 만족하다, 강좌, 것, 같다]  
1     구멍 매꾸기 위해 들어야하는 필수 강의  [구멍, 매, 꾸다, 위해, 들다, 야하다, 필수, 강의]  
2     항상 영어 3등급인데 얻어가는거 많아요                  [항상, 영어, 얻다, 많다]  
3          

In [ ]:
joo_list=[]
kang_list=[]


